# AWS Sagemaker Modeling Script

> *I used this [post](https://blog.betomorrow.com/keras-in-the-cloud-with-amazon-sagemaker-67cf11fb536) from Paul Breton and the corresponding GitHub [repo](https://github.com/Pravez/KerasSageMaker) for guidance on utilizing Keras with Sagemaker.* 

## Libraries to Import

In [ ]:
from PIL import Image
import tensorflow
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers, optimizers
from keras.layers import Dropout

import os
import gc

import sagemaker
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

import warnings
warnings.filterwarnings('ignore')

## Create Sagemaker Training Job

Create Sagemaker session and role ARN.

In [2]:
sagemaker_session = sagemaker.Session()

In [3]:
role = sagemaker.get_execution_role()

Have to define our S3 bucket name and pathway to training and validation sets, and set some hyperparameters for the model to use when calling the Keras model. 

In [4]:
bucket = "sagemaker-all-cnn"
key = "Data"
key_output = "output"                   # Path from the bucket's root to the dataset
train_instance_type='ml.m4.xlarge'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.m4.xlarge'     # The type of EC2 instance which will be used for deployment
hyperparameters={
    "learning_rate": 0.001,
    "decay": 0.0001
}

Specify locations for our data within the S3 bucket.

In [5]:
train_input_path = "s3://{}/{}/training/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

Create the actual training job. We specify additional hyperparameters, such as epochs and evaluation steps, in addition to loading the model architecture from a separate Python script (which can be found in the Model_Scripts directory in this repo) where we use the traditional Keras framework. 

In [8]:
estimator = TensorFlow(
  entry_point=os.path.join(os.path.dirname('__file__'), "Model_Scripts/Model_2x2x1x1C1D.py"),
  role=role,
  framework_version="1.12.0",               # TensorFlow's version
  hyperparameters=hyperparameters,
  training_steps=100,
  evaluation_steps=30,
  train_instance_count=1,                   # "The number of GPUs instances to use"
  train_instance_type=train_instance_type,
)

Call the .fit() method to begin the training job. Progress is tracked here, but the actual computation is done on a separate Training Jobs instance in AWS with a ml.m4.xlarge instance (a hyperparameter we can alter if the respective resource quota is satisfied). 

Depending on the model architecture, training takes from 10 mins to 1 hour with 4000 images and 100-150 epochs. 

If running this, you should see prompts in the following structure displayed over about 3 minutes. Most errors will be thrown in the last "Training in progress." step, so wait until that step is complete before navigating away.

> "Training ... 

>2020-09-22 16:09:18 Starting - Starting the training job...

>2020-09-22 16:09:20 Starting - Launching requested ML instances......

>2020-09-22 16:10:30 Starting - Preparing the instances for training...

>2020-09-22 16:11:19 Downloading - Downloading input data.........

>2020-09-22 16:12:40 Training - Training image download completed. Training in progress."

In [ ]:
print("Training ...")
estimator.fit({'training': train_input_path, 'eval': validation_input_path})

Model checkpoints and the final trained model will be uploaded to a new S3 bucket created by Sagemaker to store all models from this notebook instance. Billable seconds will also be calculated for the length of training and add to relevant totals in the AWS Billing dashboard.

## Deploy Sagemaker Model

Use .deploy() method on the trained estimator. This will take about 5 minutes to run and an AWS Endpoint will be created with the parameters defined above.

In [10]:
print("Deploying ...")
predictor = estimator.deploy(initial_instance_count=1, instance_type=deploy_instance_type)

Deploying ...
-------------!

This provides the model ID that is being hosted.

In [11]:
print("Predictor endpoint name : %s" % predictor.endpoint)

Predictor endpoint name : sagemaker-tensorflow-2020-09-29-14-14-32-855


Load not-yet-seen image to be predicted and resize it to match the model input shape.

In [110]:
import numpy as np
print("Testing endpoint ...")
all_image = load_img('./Images/ALL_Testing/419.bmp', target_size=(256, 256))
all_image_array = np.array(all_image).reshape((1, 256, 256, 3))

Testing endpoint ...


Returns a JSON object where <'float_val': [1.0, 0.0]> represents the predicted class for the loaded image. The left value is the positive class, leukemia, and the right value is the negative class, normal.

In [111]:
print(predictor.predict({'inputs_input': all_image_array}))

{'outputs': {u'dense_1': {'dtype': 1, 'float_val': [1.0, 0.0], 'tensor_shape': {'dim': [{'size': 1L}, {'size': 2L}]}}}, 'model_spec': {'signature_name': u'serving_default', 'version': {'value': 1601390206L}, 'name': u'generic_model'}}
